In [ ]:
from qiskit import QuantumCircuit

from mqt.qecc import CSSCode, StabilizerCode
from mqt.qecc.circuit_synthesis import gate_optimal_verification_circuit, heuristic_prep_circuit

For a $[[n,k,d]]$ stabilizer code we can check for X- and Z-errors with transversal CNOTs using a single $2n$ ancilla state. Half the qubits are then used as the control in a transversal CNOT and the other half as the target of another transversal CNOT. To ensure that these applications of CNOTs don't disturb the logical state we have to make sure the stabilizers propagate correctly from the state to the ancilla and vice-versa. 

Let $S$ be the stabilizer generators of an $[[n, k, d]]$ stabilizer code and $H_S = (H_X \mid H_Z) \in \mathbb{Z}_2^{(n-k)\times 2n}$ be symplectic representation of $S$. Furthermore, let $S_X = \{X^{\otimes H_X[i, :]} \mid 1 \leq i \leq (n-k)$ where $X^{\otimes v}$ is the stabilizer that acts as a Pauli $X$ on qubit $j$ if $v[j] = 1$ and as identity otherwise. $S_Z$ is defined analogous.

Then we define the $2n$-qubit state $\ket{\psi}$ as the state stabilized by $S_\mathit{anc} = \{s_X\otimes (H^{\otimes n} s_Z H^{\otimes n}) \mid s_X \in S_X, s_Z \in S_Z\}$ and $2n-|S|$ $Z$-stabilizers that commute with all stabilizers in $S_\mathit{anc}$. This is a maximal CSS code (a code without any logicals). We can synthesize such states using `qecc`.

Let's synthesize this state for the five-qubit code:

In [ ]:
# Define the code:
five_qubit_code = StabilizerCode(["XZZXI", "IXZZX", "XIXZZ", "ZXIXZ"])

# Construct the maximal CSS code. No Z-stabilizers need to be given since they are uniquely determined.
combined = CSSCode(3, Hx=five_qubit_code.symplectic_matrix)

# Synthesize the state prep circuit.
qc_prep = heuristic_prep_circuit(combined)

qc_prep.circ.draw(output="mpl", scale=0.5, initial_state="zero")

This ancilla state can be used for syndrome extraction using the following construction:

In [ ]:
qc = QuantumCircuit(5)

qc = qc_prep.circ.tensor(qc)
qc.barrier()
qc.cx(range(5, 10), range(5))
qc.barrier()
qc.h(range(10, 15))
qc.cx(range(5), range(10, 15))

qc.draw(output="mpl", scale=0.5, fold=False)

One can verify that, by construction, the stabilizers propagate correctly. For example, the stabilizer $XXZIZ$ on the data qubits propagates to $XXIII|IIZIZ$ on the ancilla state, which is a stabilizer by construction of $S_\mathit{anc}$. The other stabilizers of the ancilla state are all Z-type stabilizers, which do not propagate at all (the Hadamards on the last 5 qubits turn the Zs into X which commute through the target of the CNOTs).

Before using such an ancilla state for syndrome extraction, one might want to verify that no errors have propageted through the CNOTs during preparation. `qecc` has functionality for this as well.

In [ ]:
qc_prep_ver = gate_optimal_verification_circuit(qc_prep)

qc_prep_ver.draw(output="mpl", scale=0.5, fold=False)